![cactus](http://cactuscode.org/global/images/cactuslogo.png)
# Tutorial 2: Reworking Symmetry Boundary Conditions

In [ ]:
import os
os.environ["PATH"]="/opt/conda/envs/python2/bin:"+os.environ["PATH"]
!python --version

If you still have the Tutorial directory from the first tutorial, you do not need to run these commands. If you need to rebuild the toolkit from scratch, you can run this without needing anything from the previous tutorial.

In [ ]:
%cd ~/
%mkdir Tutorial
%cd ~/Tutorial
!curl -kLO https://raw.githubusercontent.com/gridaphobe/CRL/ET_2018_02/GetComponents
!chmod a+x ./GetComponents
!echo no|./GetComponents --parallel ~/PreSyncTutorial/thorns.th
%cd ~/Tutorial/CactusPre
!./simfactory/bin/sim setup-silent
%cd ~/Tutorial/CactusPre
!time ./simfactory/bin/sim build -j 2 --thornlist=~/Tutorial/CactusPre/thornlists/thorns.th

<h1>!Title!</h1>

In the first tutorial, we successfully updated a thorn to use PreSync. This demonstrated two different update types:
1. updating a thorn that doesn't include boundary routines
    1. add READ/WRITE declarations to scheduled functions
    2. place all old boundary scheduling into a legacy if statement
    3. add scheduling of BC selection routines in the PreSync_Selection group
    4. change DECLARE_CCTK_ARGUMENTS to the new macro and include the cctk_Arguments_Checked.h header file
2. Updating a boundary condition registered and applied using the Boundary thorn
    1. apply the changes for a standard thorn
    2. include the PreSync.h header file in the files with the boundary condition and registration functions

The second case has an important qualifier: it was specifically for boundary conditions which use Boundary to apply their boundary conditions. How do we change a boundary routine which doesn't use Boundary? In addition to this, there exists another less obvious case to consider. Symmetry boundary conditions (scheduled in BoundaryConditions group in Boundary) need to be applied to all variables, unlike physical boundary conditions. This means that PreSync needs to use a different system to apply them than the physical boundary conditions.

in this tutorial, we examine the symmetry boundary condition registered by the thorn CartGrid3D thorn and how this thorn is changed to use PreSync. To start, we need to access CartGrid3D without the PreSync changes. To do this, we will copy the master branch into the Tutorial directory so we may compare more easily inside this notebook.

In [ ]:
%cd ~/Tutorial/CactusPre/repos/cactusbase
!git checkout master
%cp -r CartGrid3D ~/Tutorial/
!git checkout presync

Below, we use diff to see how the schedule.ccl changed for PreSync. The first difference is simply a renamed function, so we can ignore that change. Several READ/WRITE declarations follow, as we would expect. The last change is different. Originally, symmetry boundary conditions were applied by the Boundary thorn. Symmetry functions were registered with Symbase, and then the ApplyBC function was scheduled inside te BoundaryConditions group provided by the Boundary thorn. This scheduled function should only run if PreSync is not active, so it checks the value of use_psync. In addition, the name changes from CartGrid3D_ApplyBC to Old_CartGrid3D_ApplyBC. This is because the argument lists are different for the two functions, so they must be declared separately. We will see the details of this change later.

In [ ]:
!diff ~/Tutorial/CactusPre/repos/cactusbase/CartGrid3D/schedule.ccl ~/Tutorial/CartGrid3D/schedule.ccl

Next, we need to look at the interface.ccl. Below, we can see that a new function has been added named Boundary_RegisterSymmetryBC. This is a new function provided by Boundary2 to register symmetry boundary conditions with PreSync. The implementation is very similar to the Boundary_RegisterPhysicalBC provided by Boundary and Boundary2. Any thorn which provides symmetry boundary conditions must use this function to properly interface with PreSync.

In [ ]:
!diff ~/Tutorial/CactusPre/repos/cactusbase/CartGrid3D/interface.ccl ~/Tutorial/CartGrid3D/interface.ccl

We can now move to the source code. Several files use the DECLARE_CCTK_ARGUMENTS macro, so these are changed to the new macro. However, this is a simple change, so we won't discuss it further. The next file with meaningful changes is RegisterSymmetries.c, which contains the function that registers the symmetry boundary condition with SymBase. The first difference is just a new name for the function, which we saw earlier in the schedule.ccl. the second change swaps out the macros. last change adds several new lines of code. We see that it is only activated if PreSync is turned on, in which case it calls the Boundary_RegisterSymmetryBC function. We could also have added a completely new function to register with PreSync, but we chose to just have a single registration function.

In [ ]:
!diff ~/Tutorial/CactusPre/repos/cactusbase/CartGrid3D/src/RegisterSymmetries.c ~/Tutorial/CartGrid3D/src/RegisterSymmetries.c

We only have two more files with changes! The first is the header file Symmetry.h. For PreSync, we included PreSync.h and also included a prototype for the ApplyBC function. The prototyping is so we can pass that function in the Boundary_RegisterSymmetryBC call.

In [ ]:
!diff ~/Tutorial/CactusPre/repos/cactusbase/CartGrid3D/src/Symmetry.h ~/Tutorial/CartGrid3D/src/Symmetry.h

The last file has the most significant changes, as it contains the symmetry function CartGrid3D_ApplyBC. Below, we extract the function declarations, including their argument lists. The old declaration takes CCTK_ARGUMENTS as its only arguments, as it is a scheduled function. The new function instead takes several arguments. These are identical to the arguments passed to physical boundary functions. This change removes several function calls within symmetry boundary condition functions, as we'll soon see.

In [ ]:
!sed -n 680,682p ~/Tutorial/CartGrid3D/src/Symmetry.c
!sed -n 680,683p ~/Tutorial/CactusPre/repos/cactusbase/CartGrid3D/src/Symmetry.c

The next two command lines extract the whole function so we can examine them in more detail. A quick comparison reveals that the old code has many more lines than the new function. Take a moment to compare the two. We explore these differences below.

In [ ]:
!sed -n 680,741p ~/Tutorial/CartGrid3D/src/Symmetry.c

In [ ]:
!sed -n 680,700p ~/Tutorial/CactusPre/repos/cactusbase/CartGrid3D/src/Symmetry.c

The first few lines of the removed code introduce some variables, which are allocated and filled in with the function Boundary_SelectedGVs. After the main code, the old code frees the memory allocations for these variables. These variables are the very ones that we pass in with the new code, and so these features needn't be inside each symmetry function individually.

Below, we see the new functions in their entirety. To update a symmetry boundary condition for PreSync, we first copy the original code and name it Old_FunctionName, to match the legacy scheduling we added earlier. Then, we remove the lines that allocate and fill the variables indices, faces, widths, and tables at the beginning of the function. We also remove the free() operations at the end.

In [ ]:
!sed -n '680,$ p' ~/Tutorial/CactusPre/repos/cactusbase/CartGrid3D/src/Symmetry.c

<table><tr><td>This work sponsored by NSF grants <a href="https://www.nsf.gov/awardsearch/showAward?AWD_ID=1550551"> OAC 1550551</a> </td><td><img src="https://www.nsf.gov/awardsearch/images/common/nsf_logo_bottom.png"></tr></table>